In [ ]:
import re
import pandas as pd

In [ ]:
NAMES = ['class', 'title', 'u1', 'authors', 'source', 'publisher', 'citations', 'abstract', 'keywords']
# DATA = 'acm_result1-6.txt'
DATA = 'acm_result1-6_unique.txt'

In [ ]:
# Check for duplicate entries.
def get_unique_lines(lines):
    _lines = sorted(lines)
    unique_lines = list()
    duplicate_line_numbers = list()
    for i in range(len(_lines) - 1):
        line = _lines[i]
        line_ = _lines[i + 1]
        if line and line not in line_:
            unique_lines.append(line)
        else:
            duplicate_line_numbers.append(i + 1)
    if _lines[-1]:
        unique_lines.append(_lines[-1])
    return unique_lines, duplicate_line_numbers

In [ ]:
def sub_count(s, to_match):
    if len(to_match) == 0:
        return 0
    if len(to_match) > len(s):
        return 0
    count = 0
    for i in range(len(s) - len(to_match) + 1):
        if s[i:i + len(to_match)] == to_match:
            count += 1
    return count


# Check for extra bar (|) characters in lines.
def get_incorrectly_separated_lines(lines, count, sep=','):
#     minus_prog = re.compile(r'^.+?(\|[^|]*?){0,' + str(count - 1) + r'}$')
    minus_lines = list()
    # 9 or more bars, with anything in-between
#     plus_prog = re.compile(r'^.+?(\|[^|]*?){' + str(count + 1) + r',}$')
    plus_lines = list()
    correct_lines = list()
    for i, line in enumerate(lines):
#         if minus_prog.match(line):
        sep_count = sub_count(line, sep)
        if sep_count < count:
            minus_lines.append(i + 1)
#         elif plus_prog.match(line):
        elif sep_count > count:
            plus_lines.append(i + 1)
        else:
            correct_lines.append(line)
    return minus_lines, plus_lines, correct_lines

In [ ]:
# Count the number of lines with `|Keywords: ` anywhere to see if it can be replaced.
def get_matching_line_count(lines, to_match):
    # keywords_prog = re.compile(r'^.+(?:\|.*){7}\|Keywords: ')
    count = 0
    for line in lines:
    #     if keywords_prog.match(line):
        if to_match in line:
            count += 1
    return count

In [ ]:
keywords_str = r'|Keywords: '
with open(DATA, 'r', encoding='utf8') as fd:
    lines = fd.readlines()
    print('Lines: {}'.format(len(lines)))
    unique_lines, duplicate_line_numbers = get_unique_lines(lines)
    print('Duplicate lines: {}'.format(len(duplicate_line_numbers)))
    minus_lines, plus_lines, correct_lines = get_incorrectly_separated_lines(unique_lines, len(NAMES) - 1, '|')
    print('Lines with too few bar (|) characters: {}'.format(len(minus_lines)))
    print('Lines with too many bar (|) characters: {}'.format(len(plus_lines)))
    keywords_count = get_matching_line_count(correct_lines, keywords_str)
    print('Lines with `|Keywords: `: {}'.format(keywords_count))

In [ ]:
print(duplicate_line_numbers)
print(minus_lines)
print(plus_lines[:min(len(plus_lines), 15)])

In [ ]:
# Replace the duplicated training data with only the unique entries, if necessary.
if len(lines) > len(correct_lines):
    DATA_UNIQUE = '_unique.'.join(DATA.split('.'))
    with open(DATA_UNIQUE, 'w+', encoding='utf8') as fd:
        for correct_line in correct_lines:
            fd.write(correct_line)
    DATA = DATA_UNIQUE

In [ ]:
data = pd.read_csv(DATA, sep='|', header=None, names=NAMES)
data.head()

In [ ]:
data = data[data.abstract.notnull()]
data.shape

In [ ]:
data.to_csv('data.csv', index=False)

### TEST

In [ ]:
test = pd.read_csv('test.txt', sep='|', header=None, names=names, encoding='latin1')